# Reformatting the Excel: Assumptions and stuff


* Copied only vital information for the data generation.
* Everything is in its own cell, so easy to import into a pandas dataframe.
* Study number five (Jorge, 2015) had measurements for 6- and 12-week checkpoints for Pain, Function and QoL. Only 12 was added to the new excel, since assumedly that is the duration of the treatment/therapy.
* O'Reilly paper was removed since they reported only the change in QoL, not the actual baseline value. Can't calculate the percentages based on just that.
* Fransen paper has reported the WOMAC scores as inverse, so 0 represents the worst status and 100 the best. Might be that others have done that too
    * _The WOMAC scores were reverse scored (100 = no pain
or difficulty, 0 = extreme pain or difficulty), so that for all outcome measures higher scores are better scores._ https://www.jrheum.org/content/jrheum/28/1/156.full.pdf (That has been adjusted to the project_excel)


* Group:
    * Used 1 - exercise and 0 - control because it is also using in the objective functions

* No of sessions:
    * Also added because it is using in the objective functions
    * 0 used for control assuming control fact do not have any sessions

* Age:
    * several different "value types" in paranthesis
    * for some there is the +/- notation: not sure what that means (another notation for SD it turns out)
    * for those cells with input like 71.9 (69.3, 74.6), in the paranthesis seems to be the 95% CI (only checked Cheung paper)
    * for some there is mean (SD) notation used, SD being standard deviation
    * According to the Wallis paper age structure may be Age_Mean_value (Age_SD)
    * According to the Evcik paper the +/- notation follows SD

* BMI:
    * same issues as with age


* Study participants/subjects:
    * for simplicity's sake, it was assumed that all participants finished the therapies/participating in the study.
        * (In reality this is not true, but cutting corners here a little bit, this is already getting out of hand)
    * Do we need to keep Male and Female separate?  - _I don't think so. The information is not used in any of the objectives or in the data later, all the numbers are for the genders combined. Ideally, we should take the gender into account, but I think it is just complicating matters at this point. -J_

* Pain levels:
    * only the womac score included
    * unless otherwise mentioned, it was assumed that the score was 0-20
    * notice: not all were in the same scale, so needed to scale
    * WOMAC (35 pain) don't know what this means and can't get access to the article to check so not doing anything to the values --> is foin
    * 'changes' was interpreted to mean how much the original womac value changed, so the 'after' value would baseline+change.


* Functionality
    * can't get access to check what womac fc 85 means so leaving that as is (since googling didn't help either) --> again, is foin

    
* QoL
    * different tests were used to test quality of life. If the scales were same, it was assumemd that they are comparable as is.
    * everything was scaled to 0-100
    * assuming that NHP emotional reaction is scaled from 0-100
    * should maybe check if all scores are the "same order", like 0 is worst in all and 100 best. (Because for example for NHP: "the total score for each domain is 100 where a score of 0 indicates good subjective health status and 100 indicates poor subjective health status." and I think SF-36 might use the scale the other way, 0 being worst and 100 being best)
        * assuming 100 is the best score and 0 the worst
        * I inversed the last study's QoL scores since they were the wrong way around (0 being the best and 100 worst)
        * assuming others are OK, since the numbers make sense (they are rising after intervention/with time)
    * Also for one paper there is no values, what should be done with that? (marked in grey)
        * _This study has been dropped._


    * **In the final report we shouold mention that several different metrics were used and that they might not be comparable**
        * Although in theory, since we are comparing the change (or the percentage of improvement), they might not be that different?


Color coding for the half finished table:
* pink: need to somehow turn values into standard deviation
    * *KNN imputation was used, right?*
* green: need to find out what mm means and turn the value into scores (need to read the source paper)
    * still don't know what mm means, but those were just another scale



_TODO: add table columns and explain them_

# Data Preprocessing

Handling missing values using KNN imputation


*I commented it for now, because I don't have the file in my gdrive, maybe you could update it so that it will work here too? -J*

In [1]:
# import numpy as np
# import pandas as pd
# from sklearn.impute import KNNImputer

# #Access files from Google Drive
# from google.colab import drive
# drive.mount('/content/gdrive')

# df = pd.read_excel("/content/gdrive/MyDrive/TIES5830/project_excel.xlsx", header=0)

# print("Dataset:")
# print(df.head())

In [2]:
# # Display the number of missing values in each column
# print("Missing Values in Each Column:")
# print(df.isnull().sum())

In [3]:
# # Identify numeric columns
# numeric_columns = df.select_dtypes(include=[float, int]).columns
# print("Numeric Columns:", numeric_columns)

In [4]:
# # Create a DataFrame with only numeric columns for KNN imputation
# numeric_df = df[numeric_columns]

# # Initialize KNN Imputer
# imputer = KNNImputer(n_neighbors=3)  # Choose a suitable number of neighbors

# # Apply KNN imputation to the numeric columns
# imputed_numeric_data = imputer.fit_transform(numeric_df)

# # Convert back to a DataFrame with the original numeric column names
# imputed_numeric_df = pd.DataFrame(imputed_numeric_data, columns=numeric_columns)

In [5]:
# # Reattach non-numeric columns to the imputed data
# imputed_df = pd.concat([df[['reference']], imputed_numeric_df], axis=1)

# print("Imputed Data:")
# print(imputed_df.head)

# Data generation


* Find a method that can generate artificial data based on mean and variance
* If a method cannot be found, need to assume distriubtion
    * Compare that to some existing data (if there is such)
        * if looks good, run with it
        * but what if it does not look good? And if there is existing data, wouldn't it be easier to generate synthetic data based on that? There's ways to generate "anonymous" (synthetic) data from existing patient information, so that no patient confidiality is broken.

## Need to generate data for


* Age
    * assumption: distribution probably skewed to the older side, since it is knee OA (older people more likely to have it)
    * modeled as normal distribution, should maybe skew it a bit to the older side?
* BMI
    * Knee osteoarthiritis and bmi connected, could use that information?  https://www.sciencedirect.com/science/article/abs/pii/S1297319X11001370
    * modeled as a normal distribution, should skew it somehow maybe?
* Pain (baseline)
    * modeled as normal distribution for now (same for all the rest - should probably think more)
* Physical Functionality (baseline)
* QoL (baseline)
* Pain (after intervention)
* Physical Functionality (after intervention)
* QoL (after intervention)

In [6]:
# Importing the data
import pandas as pd

# Importing the excel into a dataframe
df = pd.read_excel('project_excel.xlsx', index_col='reference')

## Data generation: normally distributed


All generated data is stored in a dictionary, where the first column in excel (or the df index row) is the key and its value is another dataframe of the form


{ <br>
        'age': data_age, <br>
        'bmi': data_bmi, <br>
        'patients_num': num_of_participants,<br>
        'pain_before': pain_before_data,<br>
        'pain_after': pain_after_data,<br>
        'functionality_before': func_before_data,<br>
        'functionality_after': func_after_data,<br>
        'QoL_before': qol_before_data,<br>
        'QoL_after': qol_after_data<br>
}

### <span style="color:red">NOTE:</span>
<span style="color:red"> This is a very naive approach and does not work well at all. </span> BUT it is enough to start working on the surrogates. We need to be cleverer if we want usable results. -J





**Reminder:**
 * pain on scale 0-20 (0 meaning no pain and being the best value)
    * so pain_after should be assumed to be smaller than pain_before and calculating the percentage as (before-after)/before * 100
 * function 0-68 (0 being the best)
    * percentage calculated as above
 * QoL on scale 0-100, the bigger the better.
    * value is assumed to get higher after intervention/with time, so calculating the percentage as (after-before)/before * 100

*Note: If the percentage values are negative, it means the improvement has been negative.*

In [7]:
import numpy as np

# Function that takes the whole column as input, divides it into pairs with the relevant mean and std
# except the number of participants in the study is just an integer.
# Then generates the data based on the above distributions and returns it in a dictionary
def generate_data(info):
    age = (info['age_mean'], info['age_std'])
    bmi = (info['bmi_mean'], info['bmi_std'])
    num_of_participants = int(info['num_of_subjects'])
    pain_before = (info['pain_before_mean'], info['pain_before_std'])
    pain_after = (info['pain_after_mean'], info['pain_after_std'])
    func_before = (info['func_before_mean'], info['func_before_std'])
    func_after = (info['func_after_mean'], info['func_after_std'])
    qol_before = (info['QoL_before_mean'], info['QoL_before_std'])
    qol_after = (info['QoL_after_mean'], info['QoL_after_std'])

    # Generating the data
    data_age = np.random.normal(age[0], age[1], num_of_participants)
    data_bmi = np.random.normal(bmi[0], bmi[1], num_of_participants)
    pain_before_data = np.random.normal(pain_before[0], pain_before[1], num_of_participants)
    pain_after_data = np.random.normal(pain_after[0], pain_after[1], num_of_participants)
    func_before_data = np.random.normal(func_before[0], func_before[1], num_of_participants)
    func_after_data = np.random.normal(func_after[0], func_after[1], num_of_participants)
    qol_before_data = np.random.normal(qol_before[0], qol_before[1], num_of_participants)
    qol_after_data = np.random.normal(qol_after[0], qol_after[1], num_of_participants)

    return {
        'age': data_age,
        'bmi': data_bmi,
        'patients_num': num_of_participants,
        'pain_before': pain_before_data,
        'pain_after': pain_after_data,
        'func_before': func_before_data,
        'func_after': func_after_data,
        'QoL_before': qol_before_data,
        'QoL_after': qol_after_data
    }


In [8]:
# A function for creating csv files for each exercise and control group
import csv
import os

def write_to_csv(data, filename, folder):
    os.makedirs(folder, exist_ok=True)
    filepath = os.path.join(folder, filename)

    with open(filepath, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['age', 'bmi', 'pain_before', 'pain_after',
                         'func_before', 'func_after', 'QoL_before', 'QoL_after'])

        for i in range(data['patients_num']):
            writer.writerow([data['age'][i], data['bmi'][i],
                             data['pain_before'][i], data['pain_after'][i],
                             data['func_before'][i], data['func_after'][i],
                             data['QoL_before'][i], data['QoL_after'][i]])


In [9]:
# # This piece of code generates the data. Now commented because it obviously overwrites the csv files each run
# # and we do not wish to do that each time.

# # Importing the excel into a dataframe
# df = pd.read_excel('project_excel.xlsx', index_col='reference')

# # This is the folder where the data is stuffed. The folder doesn't have to exist,
# # if it does not exist, the folder will be created
# folder = 'data_initial'

# data_dict = {} # Storing all generated data here
# for index, row in df.iterrows():
#     # Generating data for each row in the df and adding that to the dict with the row index being the key
#     data_dict[index] = generate_data(row)
#     filename = f"data_{index}.csv"
#     write_to_csv(data_dict[index], filename, folder)

## <span style="color:red">PROBLEMS WITH THE DATASETS</span>

* Negative values in the objective function data, that should not be allowed to happen since the values aren't realistic.
* Improvement rates are bloody ridiculous, they are not realistic at all. I think it is because of the negative values in the data, they are messing with the improvement calculations. Need to weed out the negative values and perhaps introduce some sort of correlation between the before and after levels.

### CSV shenanigans

In [10]:
# Function for importing the data from .csv files, returns it as a dictionary of dataframes
# Takes the folder where .csv files are located as argument. 
# If the folder does not exist, sucks to be you.
def import_data_from_csv(directory):
    df_dicts = {}

    # Looping the files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename) # path to the file
            df = pd.read_csv(filepath)

            # Calculating the improvement percent for each patient (pain, function and QoL)
            # Not sure if this is needed, but it is useful when looking at the "realness" of data
            # and comparing the results to the expectation.

            # Commented for now, in order to avoid pissin' in the StandardScaler's cereal. 
            # df['pain_improvement'] = (df['pain_before'] - df['pain_after'])/df['pain_before'] * 100
            # df['func_improvement'] = (df['func_before'] - df['func_after'])/df['func_after'] * 100
            # df['QoL_improvement'] = (df['QoL_after'] - df['QoL_before'])/df['QoL_before'] * 100
            
            # Key is the filename without the .csv ending
            df_dicts[filename[:-4]] = df
    return df_dicts

In [11]:
# Here importing the data from csv files into dataframes

directory = "./data_initial"
dataframes_dict = import_data_from_csv(directory)

In [12]:
dataframes_dict['data_an_2008_control'].head()

,age,bmi,pain_before,pain_after,func_before,func_after,QoL_before,QoL_after
0,55.994429,31.823120,3.945255,7.994302,15.065072,0.104071,86.578920,81.224085
1,57.950491,26.672613,3.954523,5.206808,19.645646,37.560577,42.440022,68.358751
2,65.929268,26.114501,5.171387,12.147902,11.366587,43.776695,85.184797,56.680762
3,78.306072,24.260916,7.058493,6.334312,21.053131,23.891605,61.233277,62.361680
4,58.472637,23.716733,4.908522,7.337675,14.024951,5.947713,67.839827,63.052998


## Data generation: GAN

*Again, commented this the GAN stuff, since we don't wanna be creating new files each time we run the notebook. Also, the gdrive thing don't work on my end.. :D -J*

In [13]:
# #!pip install numpy pandas tensorflow

# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense
# from tensorflow.keras.models import Model
# import os

# #Access files from Google Drive
# from google.colab import drive
# drive.mount('/content/gdrive')

# df = pd.read_excel("/content/gdrive/MyDrive/project_excel.xlsx", header=0)

In [14]:
# # Define GAN components
# def build_generator(latent_dim, output_dim):
#     model_input = Input(shape=(latent_dim,))
#     x = Dense(64, activation='relu')(model_input)
#     x = Dense(128, activation='relu')(x)
#     x = Dense(output_dim, activation='linear')(x)
#     model = Model(model_input, x)
#     return model

# def build_discriminator(input_dim):
#     model_input = Input(shape=(input_dim,))
#     x = Dense(128, activation='relu')(model_input)
#     x = Dense(64, activation='relu')(x)
#     x = Dense(1, activation='sigmoid')(x)
#     model = Model(model_input, x)
#     return model

# def build_gan(generator, discriminator):
#     discriminator.compile(optimizer='adam', loss='binary_crossentropy')
#     discriminator.trainable = False
#     gan_input = Input(shape=(latent_dim,))
#     generated_data = generator(gan_input)
#     gan_output = discriminator(generated_data)
#     gan = Model(gan_input, gan_output)
#     gan.compile(optimizer='adam', loss='binary_crossentropy')
#     return gan

In [15]:
# # Training function
# def train_gan(generator, discriminator, gan, data, epochs=50, batch_size=16):
#     for epoch in range(epochs):
#         # Train discriminator
#         noise = np.random.normal(0, 1, (batch_size, latent_dim))
#         generated_data = generator.predict(noise)

#         real_data = data[np.random.randint(0, data.shape[0], batch_size)]
#         combined_data = np.vstack((real_data, generated_data))
#         labels = np.array([1] * batch_size + [0] * batch_size)

#         d_loss = discriminator.train_on_batch(combined_data, labels)

#         # Train generator
#         noise = np.random.normal(0, 1, (batch_size, latent_dim))
#         misleading_labels = np.array([1] * batch_size)

#         g_loss = gan.train_on_batch(noise, misleading_labels)

#         if epoch % 100 == 0:
#             print(f'Epoch {epoch} - Discriminator Loss: {d_loss}, Generator Loss: {g_loss}')

# # Define parameters
# latent_dim = 10
# output_dim = 10

# # Create directories for saving datasets
# os.makedirs("synthetic_datasets", exist_ok=True)

# # Iterate over each reference to generate synthetic datasets
# for _, row in df.iterrows():
#     ref = row['reference']
#     num_subjects = row['num_of_subjects']

#     real_data = np.array([
#         np.full(num_subjects, row['group']),
#         np.full(num_subjects, row['no_of_sessions']),
#         np.random.normal(row['age_mean'], row['age_std'], num_subjects),
#         np.random.normal(row['bmi_mean'], row['bmi_std'], num_subjects),
#         np.random.normal(row['pain_before_mean'], row['pain_before_std'], num_subjects),
#         np.random.normal(row['pain_after_mean'], row['pain_after_std'], num_subjects),
#         np.random.normal(row['func_before_mean'], row['func_before_std'], num_subjects),
#         np.random.normal(row['func_after_mean'], row['func_after_std'], num_subjects),
#         np.random.normal(row['QoL_before_mean'], row['QoL_before_std'], num_subjects),
#         np.random.normal(row['QoL_after_mean'], row['QoL_after_std'], num_subjects)
#     ]).T

#     generator = build_generator(latent_dim, output_dim)
#     discriminator = build_discriminator(output_dim)
#     gan = build_gan(generator, discriminator)

#     train_gan(generator, discriminator, gan, real_data, epochs=1000, batch_size=16)

#     noise = np.random.normal(0, 1, (num_subjects, latent_dim))
#     synthetic_data = generator.predict(noise)

#     synthetic_df = pd.DataFrame(synthetic_data, columns=[
#         'group', 'no_of_sessions', 'age', 'bmi', 'pain_before', 'pain_after', 'func_before', 'func_after', 'QoL_before', 'QoL_after'
#     ])

#     synthetic_df['group'] = synthetic_df['group'].round().astype(int)
#     synthetic_df['no_of_sessions'] = synthetic_df['no_of_sessions'].round().astype(int)

#     synthetic_df.to_csv(f"synthetic_datasets/{ref}_synthetic.csv", index=False)
#     print(f"Generated synthetic dataset for {ref}")

# print("All synthetic datasets have been generated.")

# Data generation: normally distributed with correlations



## <span style="color:red">NOTE</span>
<span style="color:red">This will be left half done, since it was done in parallel in case GAN did not render good results. It did tho, so no need to finish this!</span> 

The idea behind this was to have some "proper" data with correlations/covariances, since the initial version (random, normally distributed) was garbage.

In [16]:
import numpy as np

# Function that takes the whole column as input, divides it into pairs with the relevant mean and std
# except the number of participants in the study is just an integer.
# Then generates the correlated data using the correlation matrix generated anew for each generation round
def generate_data_with_correlation(info):
    age = (info['age_mean'], info['age_std'])
    bmi = (info['bmi_mean'], info['bmi_std'])
    num_of_participants = int(info['num_of_subjects'])
    pain_before = (info['pain_before_mean'], info['pain_before_std'])
    pain_after = (info['pain_after_mean'], info['pain_after_std'])
    func_before = (info['func_before_mean'], info['func_before_std'])
    func_after = (info['func_after_mean'], info['func_after_std'])
    qol_before = (info['QoL_before_mean'], info['QoL_before_std'])
    qol_after = (info['QoL_after_mean'], info['QoL_after_std'])


    num_samples = num_of_participants

    # The mean values of samples
    mu = np.array([age[0], bmi[0], pain_before[0], func_before[0], qol_before[0], pain_after[0], func_after[0], qol_after[0]])

    # The covariance matrix. Right now the values are randomly picked. Should be cleverer and also maybe make different
    # correlation matrixes for each dataset/study?
    r = np.array([
    [1.0, 0.1, 0.1, 0.15, 0.15, -0.2, -0.3, -0.3],    # Variance of age
    [0.1, 1.0, 0.1, 0.15, 0.15, -0.2, -0.3, -0.3],    # Variance of BMI
    [0.1, 0.1, 1.0, 0.4, 0.4, -0.3, -0.4, -0.4],      # Covariance between age and pain_before, func_before, qol_before
    [0.15, 0.15, 0.4, 1.0, 0.7, -0.5, -0.6, -0.7],    # Covariance between BMI and pain_before, func_before, qol_before
    [0.15, 0.15, 0.4, 0.7, 1.0, -0.5, -0.6, -0.7],    # Variance of pain_before, func_before, qol_before
    [-0.2, -0.2, -0.3, -0.5, -0.5, 1.0, 0.8, 0.7],    # Covariance between age and pain_after, func_after, qol_after
    [-0.3, -0.3, -0.4, -0.6, -0.6, 0.8, 1.0, 0.9],    # Covariance between BMI and pain_after, func_after, qol_after
    [-0.3, -0.3, -0.4, -0.7, -0.7, 0.7, 0.9, 1.0]     # Variance of pain_after, func_after, qol_after
    ])



    # Generate the random samples.
    rng = np.random.default_rng()
    y = rng.multivariate_normal(mu, r, size=num_samples)

    print(y)

    # Generating the data
    data_age = np.random.normal(age[0], age[1], num_of_participants)
    data_bmi = np.random.normal(bmi[0], bmi[1], num_of_participants)
    pain_before_data = np.random.normal(pain_before[0], pain_before[1], num_of_participants)
    pain_after_data = np.random.normal(pain_after[0], pain_after[1], num_of_participants)
    func_before_data = np.random.normal(func_before[0], func_before[1], num_of_participants)
    func_after_data = np.random.normal(func_after[0], func_after[1], num_of_participants)
    qol_before_data = np.random.normal(qol_before[0], qol_before[1], num_of_participants)
    qol_after_data = np.random.normal(qol_after[0], qol_after[1], num_of_participants)

    # Calculating the improvement percent for each patient (pain, function and QoL)
    pain_improvement = (pain_before_data - pain_after_data)/pain_before_data * 100
    func_improvement = (func_before_data - func_after_data)/func_before_data * 100
    qol_improvement = (qol_after_data - qol_before_data)/qol_before_data * 100

    return {
        'age': data_age,
        'bmi': data_bmi,
        'patients_num': num_of_participants,
        'pain_before': pain_before_data,
        'pain_after': pain_after_data,
        'pain_improvement': pain_improvement,
        'func_before': func_before_data,
        'func_after': func_after_data,
        'func_improvement': func_improvement,
        'QoL_before': qol_before_data,
        'QoL_after': qol_after_data,
        'QoL_improvement': qol_improvement
    }


In [17]:
# Importing the excel into a dataframe
df = pd.read_excel('project_excel.xlsx', index_col='reference')

In [18]:
# This piece of code generates the data.

# This is the folder where the data is stuffed. The folder doesn't have to exist,
# if it does not exist, the folder will be created
folder = 'data_correlated'

correlated_synthetic_data_dict = {} # Storing all generated data here
# for index, row in df.iterrows():
#     # Generating data for each row in the df and adding that to the dict with the row index being the key
#     correlated_synthetic_data_dict[index] = generate_data(row)
#     filename = f"data_{index}.csv"
#     write_to_csv(correlated_synthetic_data_dict[index], filename, folder)

In [19]:
dataframes_dict['data_an_2008_control'].head()

,age,bmi,pain_before,pain_after,func_before,func_after,QoL_before,QoL_after
0,55.994429,31.823120,3.945255,7.994302,15.065072,0.104071,86.578920,81.224085
1,57.950491,26.672613,3.954523,5.206808,19.645646,37.560577,42.440022,68.358751
2,65.929268,26.114501,5.171387,12.147902,11.366587,43.776695,85.184797,56.680762
3,78.306072,24.260916,7.058493,6.334312,21.053131,23.891605,61.233277,62.361680
4,58.472637,23.716733,4.908522,7.337675,14.024951,5.947713,67.839827,63.052998


# GAN-generated data from CSV to dictionary
Generation was originally done in another code file. Code added above 

In [20]:
# Here importing the data from csv files into dataframes

# the folder where the data is hanging out
# this will import ALL the .csv files within that folder
directory = "./GAN_Generated"
dataframes_dict_gan = import_data_from_csv(directory)


In [21]:
dataframes_dict_gan['an_2008_exercise_synthetic'].head()

,group,no_of_sessions,age,bmi,pain_before,pain_after,func_before,func_after,QoL_before,QoL_after
0,5,44,77.117580,14.539472,5.588644,5.520327,21.988897,22.629564,70.022480,73.527466
1,6,43,74.211500,13.635116,5.348004,5.092411,21.597746,21.567600,67.242170,71.313020
2,4,37,63.971367,11.288850,4.698689,4.055223,18.586061,18.242400,57.515636,61.197197
3,4,36,62.499893,12.081700,4.425854,4.508512,17.868702,18.509598,56.733430,59.383293
4,6,51,88.805970,16.424040,6.568262,5.840533,25.406647,25.530130,80.040184,84.478540


# Surrogates


Surrogates are only fit for the exercise groups, control groups are ignored fully here. 

The goal is to see if after intervention values can be predicted using age, BMI and the baseline values. (For each objective, so trying to predict pain_after with age, BMI and pain_before.) In the first version, age, BMI and baseline value were used to predict the improvement (in percentages) of the objective. 

Also not sure if the performance metric R^2 was a good choice, it was just convenient 
    --> TODO: k-cross fold validation!? Said to be just a few lines of code. Should figure that out.

- Chosen surrogates for further evaluation and testing:
    - SVM
    - Random Forest
    - Gradient Boost Regression
    

## Scaling data



* Fitting the surrogates only for the exercise groups.
* Scaling the data since some of the methods (like SVM) work better with scaled data (sensitivity to outliers etc)
    * StandardScaler was used, although it was said to be "sensitive to outliers, and the features may scale differently from each other in the presence of outliers"
    * Unsure what counts as outlier, IMO the generated data don't contain outliars that large that they would be a worry.
    * Although! The improvement rates might be a different story!! --> deleting the code that calculates the improvement in percentages, since (with the crap data) it in some cases lead to results such as 1000 and more. Should be fine.
* We are assuming that all the data is somewhat similar so just one scaler for all is enough
    *  Also need to remember to unscale the data in the end and only one scaler makes it easier!

In [22]:
# Function for scaling the data. Using same scaler for everything since the values are roughly the same with each dataset. 
# NOTE: The scaler however does change between data_initial and gan_generated!
from sklearn.preprocessing import StandardScaler

# Function for scaling the data
def scale_data(data_dict, feature_columns, target_columns):
    feature_scaler = StandardScaler()
    target_scaler = StandardScaler()

    # Concatenate all target columns
    all_targets = pd.concat([data_dict[key][target_columns] for key in data_dict.keys()], axis=0)

    # Scale features and targets together
    scaled_features = feature_scaler.fit_transform(pd.concat([data_dict[key][feature_columns] for key in data_dict.keys()], axis=0))
    scaled_targets = target_scaler.fit_transform(all_targets)

    # Update each DataFrame with scaled features and targets
    for key in data_dict.keys():
        data_dict[key][feature_columns] = scaled_features[:len(data_dict[key])]
        data_dict[key][target_columns] = scaled_targets[:len(data_dict[key])]

    return feature_scaler, target_scaler



In [23]:
features = ['age', 'bmi', 'pain_before', 'func_before', 'QoL_before']
targets = ['pain_after', 'func_after', 'QoL_after']

In [24]:
# Extracting only the dataframes about the exercise groups
ex_keys = [k for k in dataframes_dict.keys() if "exercise" in k]
exercise_dataframes_dict = {}
for key in ex_keys:
    exercise_dataframes_dict[key] = dataframes_dict[key].copy()


# Scaling the data. This will not make a copy, 
# it is scaling the original! Scaler is returned for further use.
f_scaler, t_scaler = scale_data(exercise_dataframes_dict, features, targets)

In [25]:
# Extracting only the dataframes about the exercise groups
ex_keys_gan = [k for k in dataframes_dict_gan.keys() if "exercise" in k]
exercise_dataframes_dict_gan = {}
for key in ex_keys_gan:
    exercise_dataframes_dict_gan[key] = dataframes_dict_gan[key].copy()

# Scaling GAN data, using the functions above
f_scaler_gan, t_scaler_gan = scale_data(exercise_dataframes_dict_gan, features, targets)

Now all the data has a mean of 0 and standard deviation of 1.

# Added k-fold cross validation

As suggested, k-cross fold validation was added. 

This messes up a little bit with the structure of the code, makes some parts redundant/silly. The code was built to fit a model, and return that fitted model for (possible) later use. Now that the k-fold cross validation is added, several same models are fitted to the differently splitted dataset. Right now the code returns the last one of each iteration. I'm not sure if this is stupid. It might be. It is more complicated to build the code that way, it would have been simpler (and I suppose, the same practically) to just retrain the models again. However, the code is already done and I don't really feel like modifying it, the outcome is same (I think). 

I had no strong ideas which k value I should pick so I initially chose 10. It apparently has been said to work the best in most cases and is recommended to use if one has no better ideas. However, some of the datasets are so small that the r2 score starts spitting out warnings (I suppose less than 2 data points in one set), so 5 was chosen instead. It seems like a somewhat smart idea, splitting the data 80% for training and 20% testing.

https://machinelearningmastery.com/k-fold-cross-validation/

In [26]:
splits = 5

## SVM
* **Why choose this:** Simply because I'm already familiar with it and it is simple to implement. Also it should work OK with the data.
* **Note:** Perhaps should do hyperparameter tuning but that is computationally expensive so not implemented now. Might do that if we get better data. Right now, the data is not good so hyperparameter tuning won't make the results any better so it is just a waste of time.

In [27]:
# Function for fitting SVM

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

# df is the (exercise group) dataframe being used
# cols is the columns we are the predictors (age, bmi and baseline values for pain/funct/QoL)
# y is the "target" we are trying to "predict" (should be either pain_improvement, func_improvement or QoL_improvement)
def fit_svm(df, cols, y):
    # This here is our data
    X = df[cols]
    y = df[y]

    # Adding k-fold cross validation
    kf = KFold(n_splits=splits, shuffle=True, random_state=42)
    scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        svr = svm.SVR()
        svr.fit(X_train, y_train)
        y_pred = svr.predict(X_test)
        r2 = r2_score(y_test, y_pred)    
        scores.append(r2)
    
    avg_r2 = sum(scores) / len(scores)

    return avg_r2, svr



## SVM: Initial data version

In [28]:
results_svm_dict = {}  # Storing the results and fitted surrogates here

# These are the labels, or values we are trying to precit
# This is not the cleanest way of doing it, but it is a quick fix.
options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]


for key in exercise_dataframes_dict.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, svr = fit_svm(exercise_dataframes_dict[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'svr': svr}
    results_svm_dict[key] = temp

The structure of the dictionary is a bit confusing but should work.

In [29]:
results_svm_dict['data_an_2008_exercise']

{'pain_after': {'r2': -40.726672969046234, 'svr': SVR()},
 'func_after': {'r2': -135.27724017071026, 'svr': SVR()},
 'QoL_after': {'r2': -0.7575268793833315, 'svr': SVR()}}

## SVM: GAN-generated

In [30]:
results_svm_dict_gan = {}  # Storing the results and fitted surrogates here

# These are the labels, or values we are trying to precit
# This is not the cleanest way of doing it, but it is a quick fix.
options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]

for key in exercise_dataframes_dict_gan.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, svr = fit_svm(exercise_dataframes_dict_gan[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'svr': svr}
    results_svm_dict_gan[key] = temp

In [31]:
results_svm_dict_gan['an_2008_exercise_synthetic']

{'pain_after': {'r2': 0.8055397132732599, 'svr': SVR()},
 'func_after': {'r2': 0.9694937150186445, 'svr': SVR()},
 'QoL_after': {'r2': 0.964048314402189, 'svr': SVR()}}

# Random forests



* *Why choose it:* It should work OK with the smaller datasets, overfitting shouldn't be an issue
* Note: Hyperparameter tuning might have been a good idea, but time ran out.

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def fit_random_forest(df, cols, y):
    # This here is our data
    X = df[cols]
    y = df[y]

    # Adding k-fold cross validation
    kf = KFold(n_splits=splits, shuffle=True, random_state=42)
    
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        rf_reg = RandomForestRegressor(n_estimators=5, random_state=42)
        rf_reg.fit(X_train, y_train)
        y_pred = rf_reg.predict(X_test)
        r2 = r2_score(y_test, y_pred)    
        scores.append(r2)
    
    avg_r2 = sum(scores) / len(scores)

    return avg_r2, rf_reg

## RF: Random data

In [33]:
results_forest_reg_dict = {}

options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]

for key in exercise_dataframes_dict.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, forest_reg = fit_random_forest(exercise_dataframes_dict[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'forest_reg': forest_reg}
    results_forest_reg_dict[key] = temp

In [34]:
# Get the pain_improvement predictor/surrogate function or whatever it is called
results_forest_reg_dict['data_cheung_2014_exercise']['pain_after']['forest_reg']

RandomForestRegressor(n_estimators=5, random_state=42)

# RF: GAN-data

In [35]:
results_forest_reg_dict_gan = {}

options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]

for key in exercise_dataframes_dict_gan.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, forest_reg = fit_random_forest(exercise_dataframes_dict_gan[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'forest_reg': forest_reg}
    results_forest_reg_dict_gan[key] = temp

In [36]:
# Get the pain_improvement predictor/surrogate function or whatever it is called
results_forest_reg_dict_gan['cheung_2014_exercise_synthetic']['pain_after']['forest_reg']

RandomForestRegressor(n_estimators=5, random_state=42)

# Gradient Boosting Regression



This model is effective for handling regression tasks, especially when dealing with structured tabular data, and normally it provides better performance compared to Random Forests.

In [37]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

def fit_gradient_boosting(df, cols, y):
    X = df[cols]
    y = df[y]

    # Adding k-fold cross validation
    kf = KFold(n_splits=splits, shuffle=True, random_state=42)
    
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        gb_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
        gb_reg.fit(X_train, y_train)
        y_pred = gb_reg.predict(X_test)
        r2 = r2_score(y_test, y_pred)   
        scores.append(r2)
    
    avg_r2 = sum(scores) / len(scores)

    return avg_r2, gb_reg

## GB Regression: Random data

In [38]:
results_gb_dict = {}
options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]

for key in exercise_dataframes_dict.keys():
    temp = {}
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, gb_reg = fit_gradient_boosting(exercise_dataframes_dict[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'gb_reg': gb_reg}
    results_gb_dict[key] = temp


In [39]:

results_gb_dict['data_an_2008_exercise']

{'pain_after': {'r2': -89.0059035630772,
  'gb_reg': GradientBoostingRegressor(random_state=42)},
 'func_after': {'r2': -294.3005400575695,
  'gb_reg': GradientBoostingRegressor(random_state=42)},
 'QoL_after': {'r2': -1.3818718309809137,
  'gb_reg': GradientBoostingRegressor(random_state=42)}}

## GB Regression: GAN Data

In [40]:
results_gb_dict_gan = {}
options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]
for key in exercise_dataframes_dict_gan.keys():
    temp = {}
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, gb_reg = fit_gradient_boosting(exercise_dataframes_dict_gan[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'gb_reg': gb_reg}
    results_gb_dict_gan[key] = temp


In [41]:

results_gb_dict_gan['an_2008_exercise_synthetic']

{'pain_after': {'r2': 0.9162473306035211,
  'gb_reg': GradientBoostingRegressor(random_state=42)},
 'func_after': {'r2': 0.9760646148810587,
  'gb_reg': GradientBoostingRegressor(random_state=42)},
 'QoL_after': {'r2': 0.9720286589716214,
  'gb_reg': GradientBoostingRegressor(random_state=42)}}

# Tables

Here forming tables and printing out the results to be easily viewed. 

Acquired tables were turned into markdown notation using ChatGPT. Results can be seen below

In [42]:
# Function for extracting the scores from the dictionaries
def extract_scores(results_dict, objectives, dec_places=4):

    scores = {obj: [] for obj in objectives}
    for key in results_dict:
        for obj in objectives:
            scores[obj].append(round(results_dict[key][obj]['r2'], dec_places))
    return scores


In [43]:
# Creating the table
from prettytable import PrettyTable


def create_pretty_table(therapies, svm_scores, forest_scores, gb_scores, objectives):
    table = PrettyTable()
    table.add_column('Exercise therapy', therapies)
    
    for obj in objectives:
        table.add_column(f'R2 score SVM {obj.split("_")[0]}', svm_scores[obj])
        table.add_column(f'R2 score Forest {obj.split("_")[0]}', forest_scores[obj])
        table.add_column(f'R2 score GB {obj.split("_")[0]}', gb_scores[obj])
    
    return table


In [44]:
# Crappy data

therapies = list(results_svm_dict.keys())
objectives = ['pain_after', 'func_after', 'QoL_after']

svm_scores = extract_scores(results_svm_dict, objectives)
forest_scores = extract_scores(results_forest_reg_dict, objectives)
gb_scores = extract_scores(results_gb_dict, objectives)

table_init = create_pretty_table(therapies, svm_scores, forest_scores, gb_scores, objectives)

# print(table_init)


In [45]:
# GAN-data
therapies_gan = list(results_svm_dict_gan.keys())
objectives = ['pain_after', 'func_after', 'QoL_after']

svm_scores_gan = extract_scores(results_svm_dict_gan, objectives)
forest_scores_gan = extract_scores(results_forest_reg_dict_gan, objectives)
gb_scores_gan = extract_scores(results_gb_dict_gan, objectives)

table_gan = create_pretty_table(therapies_gan, svm_scores_gan, forest_scores_gan, gb_scores_gan, objectives)

# print(table_gan)


## Random data


### Without k-fold cross validation

| Exercise therapy                | R2 score SVM pain | R2 score Forest pain | R2 score GB pain | R2 score SVM func | R2 score Forest func | R2 score GB func | R2 score SVM QoL | R2 score Forest QoL | R2 score GB QoL |
|---------------------------------|-------------------|----------------------|------------------|-------------------|----------------------|------------------|------------------|---------------------|-----------------|
| data_an_2008_exercise           | -0.6914           | -0.8982              | -1.5501          | -0.4759           | -1.1577              | -0.2932          | 0.4705           | 0.205               | -0.3348         |
| data_cheung_2014_exercise       | -0.1802           | -0.4271              | -0.543           | -0.6104           | -0.8779              | -0.7663          | -3.7761          | -6.1664             | -10.5555        |
| data_evcik_2002_exercise1       | -1.0797           | -0.6212              | -0.7551          | -0.756            | -3.4595              | -6.2033          | -0.5791          | -0.7633             | -1.5702         |
| data_evcik_2002_exercise2       | -0.8139           | -1.0763              | -1.0567          | 0.24              | -0.1246              | -0.2033          | -0.9896          | -0.414              | -0.9552         |
| data_fransen_2001_exercise1     | -0.4017           | -1.0146              | -0.4807          | -0.1789           | -0.431               | -1.0302          | -0.3398          | -0.3302             | -0.3496         |
| data_fransen_2001_exercise2     | 0.0448            | -0.32                | -0.4974          | -0.1076           | -0.1192              | -0.4381          | 0.2391           | -0.0358             | 0.008           |
| data_jorge_2015_exercise        | -0.6206           | -0.5265              | -0.9307          | -0.1671           | -0.2267              | -0.1905          | -0.5941          | -0.1543             | 0.0318          |
| data_lee_2008_exercise          | -0.2755           | -0.4396              | -1.0227          | -0.2289           | -0.7586              | -2.3562          | -1.2375          | -1.1931             | -1.4122         |
| data_lee_2009_exercise          | -5.0723           | -4.4588              | -5.5943          | -0.6224           | -0.924               | -1.7738          | -0.0384          | 0.2447              | -0.16           |
| data_salacinski_2012_exercise   | -0.7618           | -0.7855              | -0.6074          | -2.0316           | -1.8753              | -2.577           | -3.1592          | -3.4713             | -4.3074         |
| data_wallis_2017_exercise       | -0.4665           | -0.3142              | -0.0839          | -0.3792           | -0.7595              | -0.6224          | -0.228           | -0.8155             | -0.3191         |


## With k-fold cross validation

*Insert table*

## GAN-generated data

### Without k-fold cross validation


| Exercise therapy                   | R2 score SVM pain | R2 score Forest pain | R2 score GB pain | R2 score SVM func | R2 score Forest func | R2 score GB func | R2 score SVM QoL | R2 score Forest QoL | R2 score GB QoL |
|------------------------------------|-------------------|----------------------|------------------|-------------------|----------------------|------------------|------------------|---------------------|-----------------|
| an_2008_exercise_synthetic         | 0.9444            | 0.843                | 0.9129           | 0.9012            | 0.837                | 0.9607           | 0.9175           | 0.942               | 0.967           |
| cheung_2014_exercise_synthetic     | 0.4373            | 0.3909               | -0.0049          | 0.9205            | 0.9412               | 0.993            | 0.9224           | 0.9236              | 0.9731          |
| evcik_2002_exercise1_synthetic     | 0.9707            | 0.971                | 0.9237           | 0.9911            | 0.9333               | 0.986            | 0.9928           | 0.98                | 0.9872          |
| evcik_2002_exercise2_synthetic     | 0.938             | 0.9343               | 0.92             | 0.9672            | 0.9327               | 0.934            | 0.9789           | 0.9705              | 0.9457          |
| fransen_2001_exercise1_synthetic   | 0.8766            | 0.8507               | 0.9096           | 0.925             | 0.9397               | 0.9496           | 0.6551           | 0.6208              | 0.3682          |
| fransen_2001_exercise2_synthetic   | 0.4964            | 0.5691               | 0.43             | 0.9758            | 0.9746               | 0.99             | 0.9899           | 0.9945              | 0.9947          |
| jorge_2015_exercise_synthetic      | 0.7986            | 0.8563               | 0.7469           | 0.8913            | 0.8689               | 0.8795           | 0.9805           | 0.973               | 0.9831          |
| lee_2008_exercise_synthetic        | 0.8163            | 0.9184               | 0.9437           | 0.7472            | 0.8436               | 0.857            | 0.8743           | 0.9459              | 0.9398          |
| lee_2009_exercise_synthetic        | 0.9397            | 0.9064               | 0.9472           | 0.9673            | 0.6264               | 0.7381           | 0.912            | 0.8836              | 0.8267          |
| salacinski_2012_exercise_synthetic | 0.6327            | 0.3144               | 0.3275           | 0.0207            | -0.8448              | -0.4018          | 0.9288           | 0.9506              | 0.9681          |
| wallis_2017_exercise_synthetic     | 0.832             | 0.7696               | 0.7762           | 0.9279            | 0.9648               | 0.9848           | 0.9181           | 0.9662              | 0.9827          |


### With k-fold cross validation


|          Exercise therapy          | R2 score SVM pain | R2 score Forest pain | R2 score GB pain | R2 score SVM func | R2 score Forest func | R2 score GB func | R2 score SVM QoL | R2 score Forest QoL | R2 score GB QoL |
|------------------------------------|-------------------|----------------------|------------------|-------------------|----------------------|------------------|------------------|---------------------|-----------------|
| an_2008_exercise_synthetic         | 0.8895            | 0.774                | 0.9132           | 0.9438            | 0.9165               | 0.9741           | 0.9409           | 0.9102              | 0.9746          |
| cheung_2014_exercise_synthetic     | 0.1756            | 0.1753               | 0.0589           | 0.8971            | 0.8589               | 0.9492           | 0.8766           | 0.769               | 0.9268          |
| evcik_2002_exercise1_synthetic     | 0.8955            | 0.9222               | 0.9465           | 0.925             | 0.9433               | 0.9817           | 0.9313           | 0.9626              | 0.9833          |
| evcik_2002_exercise2_synthetic     | 0.6993            | 0.8696               | 0.8724           | 0.7243            | 0.878                | 0.8859           | 0.7227           | 0.9                 | 0.906           |
| fransen_2001_exercise1_synthetic   | 0.8108            | 0.8571               | 0.78             | 0.8313            | 0.9424               | 0.9356           | 0.3113           | 0.1341              | 0.264           |
| fransen_2001_exercise2_synthetic   | 0.3535            | 0.1099               | 0.1626           | 0.9716            | 0.9695               | 0.9869           | 0.9859           | 0.9917              | 0.9921          |
| jorge_2015_exercise_synthetic      | 0.3629            | 0.3748               | 0.2732           | 0.7996            | 0.8586               | 0.8757           | 0.8988           | 0.957               | 0.9678          |
| lee_2008_exercise_synthetic        | 0.81              | 0.9303               | 0.9072           | 0.8157            | 0.8078               | 0.7279           | 0.8296           | 0.9082              | 0.9181          |
| lee_2009_exercise_synthetic        | 0.798             | 0.8724               | 0.8953           | 0.7734            | 0.2987               | 0.6166           | 0.8376           | 0.6361              | 0.7979          |
| salacinski_2012_exercise_synthetic | -0.5823           | -3.2075              | -1.2542          | -0.1042           | -0.8307              | -0.5536          | 0.9498           | 0.9273              | 0.9407          |
| wallis_2017_exercise_synthetic     | 0.3657            | 0.2783               | 0.4045           | 0.9219            | 0.8444               | 0.8835           | 0.9451           | 0.8699              | 0.9218          |


## What to do with results that aren't good?

We can see above, that salacinski, for example is giving shit results. So is, for example,  cheung in pain score prediction. Not sure what should be done. In an ideal case, new surrogates would be fit and se if results would get better. If not, then a closer look would be taken into the data. Plotting should and could work to see if there is something happening that the models arent picking up on. Perhaps the k fold cross validation was dumbly implemented, perhaps the splits should have been fewer/more. Because time is almost out with this project, choosing to remove all the lines with crap results, even if that means in one objective only. Crap R2 score is here defined to be under 0.6. 

This leaves us only 5 therapies in total left for implementing the problem. This is not ideal, this way many excellent therapy alternatives are ruled out. However, workload wise it works in our favour - less things for us to implement!

We have left: 
* an
* evcik 1 and 2
* both lees (2008 and 2009)


# Picking the best performing surrogates

Here yada yada about which surrogate performed best for each dataset




### Question

Would it be better to predict after intervention scores or the improvement percentage?

After a quick experiment, it seems that (for GAN data) surrogates perform better when predicting after scores compared to predicting improvement percentage. 

If we were to predict improvement, forming the objectives would be simpler, since the predictor would give the percentage straight up. If we predict the after score, percentage would have to be calculated, but that is not such a big deal? Just needs some extra work. So rolling with the predicting after scores thing. Need to remember to yada yada about this in the final report also!

In [46]:
# Checking that the keys are the same for all results dics
print(results_svm_dict_gan.keys() == results_gb_dict_gan.keys())
print(results_svm_dict_gan.keys() == results_forest_reg_dict_gan.keys())

True
True


In [47]:
# This picks out the best performing surrogate predictor for each study
# Results stored: How?

# These are all surrogate results
results = [results_svm_dict_gan, results_forest_reg_dict_gan, results_gb_dict_gan]

# Keys are the same for all
keys = results_gb_dict_gan.keys()

# Keys to different scores/objectives
objectives = ['pain_after', 'func_after', 'QoL_after']

# Loop keys, then loop score type/objectives, THEN FINALL
# Y loop all results, check which method has the best R2 score, pick that predictor. Stuff all
best_surrogates = {'pain_after': [], 'func_after': [], 'QoL_after': [] }
for key in keys:
    for obj in objectives:
        # List comprehension: get the values
        vals = [x[key][obj] for x in results]

        # Here we choose the surrogate with the best r2 score
        best = max(vals, key=lambda x: x['r2'])
        # print(best)

        # Get the key to the surrogate model (in hidsight, should have done this in a better way earlier in code)
        key_to_surr = list(best.keys())[1]

        # Add to the correct list
        best_surrogates[obj].append({key: best[key_to_surr]})


In [48]:
# Picking out the groups for implementing the problem. 
def filter_exercise_groups(data, keywords):
    filtered_data = {key: [] for key in data.keys()}
    for key, models in data.items():
        for model_dict in models:
            for group_name in model_dict.keys():
                # Extract the part of the group name before the first underscore
                prefix = group_name.split('_')[0]
                if prefix in keywords:
                    filtered_data[key].append(model_dict)
    
    return filtered_data

In [49]:
obj_dict = filter_exercise_groups(best_surrogates, ['an', 'lee', 'evcik'])

In [50]:
obj_dict

{'pain_after': [{'an_2008_exercise_synthetic': GradientBoostingRegressor(random_state=42)},
  {'evcik_2002_exercise1_synthetic': GradientBoostingRegressor(random_state=42)},
  {'evcik_2002_exercise2_synthetic': GradientBoostingRegressor(random_state=42)},
  {'lee_2008_exercise_synthetic': GradientBoostingRegressor(random_state=42)},
  {'lee_2009_exercise_synthetic': GradientBoostingRegressor(random_state=42)}],
 'func_after': [{'an_2008_exercise_synthetic': GradientBoostingRegressor(random_state=42)},
  {'evcik_2002_exercise1_synthetic': SVR()},
  {'evcik_2002_exercise2_synthetic': GradientBoostingRegressor(random_state=42)},
  {'lee_2008_exercise_synthetic': GradientBoostingRegressor(random_state=42)},
  {'lee_2009_exercise_synthetic': GradientBoostingRegressor(random_state=42)}],
 'QoL_after': [{'an_2008_exercise_synthetic': GradientBoostingRegressor(random_state=42)},
  {'evcik_2002_exercise1_synthetic': GradientBoostingRegressor(random_state=42)},
  {'evcik_2002_exercise2_synthetic

Now we have all the best surrogates extracted, we use those to construct the objective function. 

In [51]:
data = np.array([89, 24, 8.9, 20, 65])
data = data.reshape(1, -1)
f_scaler_gan.transform(data)

c:\Users\omistaja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 1.4463658 , -0.33191303, -0.24234167, -0.36454112,  0.63095941]])

In [52]:
best_surrogates['pain_after']  # this is a list of dictionaries
surr_an = best_surrogates['pain_after'][0]['an_2008_exercise_synthetic'] # this is messy, but it works. 

# # TODO: pretty sure this is not correct, need to figure out when i have more time
# # output should change when input does but it stays the same. I don't know what the hell is going on
# # ITS ABOUT NORMALIZATION I THINK!! We need to scale the data before we add it in and the output needs to be rescaled back.

data = np.array([89, 24, 8.9, 20, 65])
data = data.reshape(1, -1)
data_scaled = f_scaler_gan.transform(data)


c:\Users\omistaja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [53]:
data = np.array([data_scaled[0][0],data_scaled[0][1], data_scaled[0][2] ])
data = data.reshape(1, -1)

# # Ensure input data is reshaped correctly (1 sample, 3 features)input_data_reshaped = np.reshape(input_data, (1, -1))

# Make the prediction
prediction = surr_an.predict(data)
print(prediction)

preds = np.array([prediction]*3)
# Need to unscale 
t_scaler_gan.inverse_transform(preds.reshape(1, -1))

[0.4517998]


c:\Users\omistaja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


array([[ 6.63948233, 25.8952995 , 75.57311854]])

# Problem formulation

### Preferences provided by the DM in the original paper
Acceptable minimum and maximum ranges for the objectives (**Can't use it exactly like this, need to modify**)

| |Costs (€) | Pain change (%) | Function change (%) | Supervised sessions | Period (weeks) |
| -------|------------|------------------|----------------------|----------------------|----------------|
| Iteration 1 | 300 | +30% | +25% | 0 | 8 |
| | 600 | +15% | +15% | 15 | 26 |
| Iteration 2  | 200 | +25% | +40% | 0 | 12 |
| | 500 | +15% | +15% | 30 | 26 |


Suggestion by P as DM (Didn't change much):

| |Costs (€) | Pain change (%) | Function change (%) | Supervised sessions | Period (weeks) |
| -------|------------|------------------|----------------------|----------------------|----------------|
| Iteration 1 | 200 | +30% | +25% | 0 | 8 |
| | 800 | +15% | +15% | 15 | 24 |
| Iteration 2  | 150 | +25% | +40% | 0 | 12 |
| | 600 | +15% | +15% | 20 | 24 |


# Reference point method

In [54]:
from desdeo_problem import variable_builder
from desdeo_problem import _ScalarObjective
from desdeo_problem import MOProblem
from desdeo_mcdm.utilities.solvers import payoff_table_method
from desdeo_mcdm.interactive.ReferencePointMethod import ReferencePointMethod

In [55]:
# Variables defined here. TODO: Figure out how to make the problem binary.

In [56]:
# Here for easy access the keys to each surrogate model in the obj_dict

an = 'an_2008_exercise_synthetic'
evcik1 = 'evcik_2002_exercise1_synthetic'
evcik2 = 'evcik_2002_exercise2_synthetic'
lee1 = 'lee_2008_exercise_synthetic'
lee2 = 'lee_2009_exercise_synthetic'

In [57]:
t_scaler_gan.inverse_transform([[0.1, 0.1, 0.1]])

array([[ 4.63582442, 22.00482052, 65.35932335]])

In [63]:
def predict_after_scores(baselines):
    predicted_after_scores = []
    obj_dict.keys()

In [59]:
# Objectives. Assuming the features have already been scaled when bringing them here!!!

# predicted after scores
predicted_after_scores = []

# Pain improvement
def objective_pain(age, bmi, baseline):
    # Making sure the input is in the correct form for the surrogates
    input = np.array([age, bmi, baseline])
    input = input.reshape(1,-1)
    return 

In [64]:
obj_dict.keys()

dict_keys(['pain_after', 'func_after', 'QoL_after'])

# Different approach (?)

In [61]:
# %pip install desdeo

In [62]:
# #I did not run the code and it seems colab has an issue of loading synthetic data

# #Just Seperate Implementation using DESDEO
# from desdeo.problem import ScalarDataProblem, ScalarMOProblem
# from desdeo.variables import variable_builder
# from desdeo.metrics import Hypervolume

# ranges_iteration_1 = [
#     (200, 800),    # Costs (€)
#     (30, 15),      # Pain change (%)
#     (25, 15),      # Function change (%)
#     (0, 15),       # Supervised sessions
#     (8, 24)        # Period (weeks)
# ]

# ranges_iteration_2 = [
#     (150, 600),    # Costs (€)
#     (25, 15),      # Pain change (%)
#     (40, 15),      # Function change (%)
#     (0, 20),       # Supervised sessions
#     (12, 24)       # Period (weeks)
# ]

# # Bit changed from orginal paper
# variable_type = ["int", "float", "float", "int", "int"]
# variable_ranges = [(150, 800), (-30, 30), (-30, 30), (0, 20), (8, 24)]

# problem_iteration_1 = ScalarDataProblem(variable_builder(variable_type, variable_ranges),
#                                         ["Costs (€)", "Pain change (%)", "Function change (%)",
#                                          "Supervised sessions", "Period (weeks)"],
#                                         ["minimize", "minimize", "minimize", "minimize", "minimize"],
#                                         variables_description=["Costs associated with the therapy",
#                                                                "Percentage change in pain",
#                                                                "Percentage change in function",
#                                                                "Number of supervised sessions",
#                                                                "Duration of therapy in weeks"],
#                                         ideal=[200, 30, 25, 0, 8],
#                                         nadir=[800, 15, 15, 15, 24],
#                                         constraints=ranges_iteration_1)

# problem_iteration_2 = ScalarDataProblem(variable_builder(variable_type, variable_ranges),
#                                         ["Costs (€)", "Pain change (%)", "Function change (%)",
#                                          "Supervised sessions", "Period (weeks)"],
#                                         ["minimize", "minimize", "minimize", "minimize", "minimize"],
#                                         variables_description=["Costs associated with the therapy",
#                                                                "Percentage change in pain",
#                                                                "Percentage change in function",
#                                                                "Number of supervised sessions",
#                                                                "Duration of therapy in weeks"],
#                                         ideal=[150, 25, 40, 0, 12],
#                                         nadir=[600, 15, 15, 20, 24],
#                                         constraints=ranges_iteration_2)


# problems = [problem_iteration_1, problem_iteration_2]
# scalar_mo_problem = ScalarMOProblem(problems)

# hypervolume_metric = Hypervolume(scalar_mo_problem)

# # Can change algorithm. Just used NSGA2
# from desdeo.optimizers.NSGA2 import NSGA2

# optimizer = NSGA2(scalar_mo_problem)

# n_iterations = 100
# approximation = optimizer.minimize(n_iterations)

# for i, problem in enumerate(problems):
#     print(f"Iteration {i+1}:")
#     print(f"  Number of solutions: {len(approximation.objectives[i])}")
#     print(f"  Hypervolume: {hypervolume_metric(approximation.objectives[i])}")